In [1]:
from glob import glob
import SimpleITK as sitk

In [2]:
LUNA_DATASET_PATH = "/mnt/DATASETS/LUNA/"
EXPORT_IMAGE_FORMAT = ".nii.gz"

In [3]:
def get_id_luna_image(path):
    return path.split("/")[-1].split(".mhd")[0]

def load_luna_image(path):
    return sitk.ReadImage(path)

In [20]:
def calculate_transformation_map(fixed_image, moving_image, moving_image_mask):
    """Registers moving_image to fixed_image and applies the resulting to transformation to moving_image_mask,
    which is returned as a new image.
    Performing a multiresolution image registration, with translation, affine and non-rigid transformations.
    """
    # Apply the mask to the luna image. We don't want to register the whole ribcage, just the lungs
    mask_image_filter = sitk.MaskImageFilter()
    masked_moving_image = mask_image_filter.Execute(moving_image, moving_image_mask)
    
    # Registration parameters
    parameterMapVector = sitk.VectorOfParameterMap()
    parameterMapVector.append(sitk.GetDefaultParameterMap("translation"))
    parameterMapVector.append(sitk.GetDefaultParameterMap("affine"))
    parameterMapVector.append(sitk.GetDefaultParameterMap("bspline"))
    
    # Perform registration
    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(fixed_image)
    elastixImageFilter.SetMovingImage(masked_moving_image)
    elastixImageFilter.SetParameterMap(parameterMapVector)
    elastixImageFilter.Execute()
    
    # Return transformation parameter map
    return elastixImageFilter

def calculate_average_image_transformation(image, transformations):
    transformixFilter = sitk.TransformixImageFilter()
    transformixFilter.SetMovingImage(image)
    transformixFilter.SetTransformParameterMap(transformations[0].GetTransformParameterMap())
    for transformation_tuple in transformations[1:]:
        for transformation in transformation_tuple.GetTransformParameterMap():
            transformixFilter.AddTransformParameterMap(transformation)
    transformixFilter.Execute()
    return transformixFilter.GetResultImage()

In [5]:
luna_images_paths = glob(LUNA_DATASET_PATH + "subset*/*.mhd")

luna_segmentation_paths = glob(LUNA_DATASET_PATH + "seg-lungs-LUNA16/*.mhd")
luna_segmentation_paths_by_id = {get_id_luna_image(path): path for path in luna_segmentation_paths}

print("LUNA dataset has %d images" % (len(luna_images_paths)))

LUNA dataset has 742 images


In [6]:
fixed_image_paths = luna_images_paths[:2]
moving_image_path = luna_images_paths[-1]

num_iterations = 1

In [7]:
id_luna = get_id_luna_image(moving_image_path)
reference_img = load_luna_image(moving_image_path)
reference_img_mask = load_luna_image(luna_segmentation_paths_by_id[id_luna])

for i in range(num_iterations):
    print("Starting iteration %d of the creation of a population atlas" % (i+1,))
    transformations = []
    for j, fixed_image_path in enumerate(fixed_image_paths):
        print("Registering image %d out of %d" % (j+1, len(fixed_image_paths)))
        fixed_img = load_luna_image(fixed_image_path)
        %time transformation = calculate_transformation_map(fixed_img, reference_img, reference_img_mask)
        transformations.append(transformation)
    print("Applying average transformation to reference image")
    %time reference_img = calculate_average_image_transformation(reference_img, transformations)
    print("Applying average transformation to reference image mask")
    %time reference_img_mask = calculate_average_image_transformation(reference_img_mask, transformations)
    # Reduce mask byte depth
    reference_img_mask = sitk.Cast(reference_img_mask, sitk.sitkInt8)
    # Save intermediate results
    id_atlas_luna = str(i) + "_" + id_luna
    img_atlas_path = LUNA_DATASET_PATH + "atlas/img_" + id_atlas_luna + EXPORT_IMAGE_FORMAT
    sitk.WriteImage(reference_img, img_atlas_path)
    mask_atlas_path = LUNA_DATASET_PATH + "atlas/mask_" + id_atlas_luna + EXPORT_IMAGE_FORMAT
    sitk.WriteImage(reference_img_mask, mask_atlas_path)
    print("Saved reference image for iteration %d out of %d" % (i+1, num_iterations))

Starting iteration 1 of the creation of a population atlas
Registering image 1 out of 2
CPU times: user 9min 25s, sys: 16.7 s, total: 9min 42s
Wall time: 5min 7s
Registering image 2 out of 2
CPU times: user 9min 4s, sys: 17.4 s, total: 9min 22s
Wall time: 4min 54s
Applying average transformation to reference image


NotImplementedError: Wrong number or type of arguments for overloaded function 'TransformixImageFilter_SetTransformParameterMap'.
  Possible C/C++ prototypes are:
    itk::simple::TransformixImageFilter::SetTransformParameterMap(std::vector< std::map< std::string,std::vector< std::string,std::allocator< std::string > >,std::less< std::string >,std::allocator< std::pair< std::string const,std::vector< std::string,std::allocator< std::string > > > > >,std::allocator< std::map< std::string,std::vector< std::string,std::allocator< std::string > >,std::less< std::string >,std::allocator< std::pair< std::string const,std::vector< std::string,std::allocator< std::string > > > > > > > const)
    itk::simple::TransformixImageFilter::SetTransformParameterMap(std::map< std::string,std::vector< std::string,std::allocator< std::string > >,std::less< std::string >,std::allocator< std::pair< std::string const,std::vector< std::string,std::allocator< std::string > > > > > const)


Applying average transformation to reference image mask


NotImplementedError: Wrong number or type of arguments for overloaded function 'TransformixImageFilter_SetTransformParameterMap'.
  Possible C/C++ prototypes are:
    itk::simple::TransformixImageFilter::SetTransformParameterMap(std::vector< std::map< std::string,std::vector< std::string,std::allocator< std::string > >,std::less< std::string >,std::allocator< std::pair< std::string const,std::vector< std::string,std::allocator< std::string > > > > >,std::allocator< std::map< std::string,std::vector< std::string,std::allocator< std::string > >,std::less< std::string >,std::allocator< std::pair< std::string const,std::vector< std::string,std::allocator< std::string > > > > > > > const)
    itk::simple::TransformixImageFilter::SetTransformParameterMap(std::map< std::string,std::vector< std::string,std::allocator< std::string > >,std::less< std::string >,std::allocator< std::pair< std::string const,std::vector< std::string,std::allocator< std::string > > > > > const)


Saved reference image for iteration 1 out of 1


In [8]:
new_reference_img = calculate_average_image_transformation(reference_img, transformations)

NotImplementedError: Wrong number or type of arguments for overloaded function 'TransformixImageFilter_SetTransformParameterMap'.
  Possible C/C++ prototypes are:
    itk::simple::TransformixImageFilter::SetTransformParameterMap(std::vector< std::map< std::string,std::vector< std::string,std::allocator< std::string > >,std::less< std::string >,std::allocator< std::pair< std::string const,std::vector< std::string,std::allocator< std::string > > > > >,std::allocator< std::map< std::string,std::vector< std::string,std::allocator< std::string > >,std::less< std::string >,std::allocator< std::pair< std::string const,std::vector< std::string,std::allocator< std::string > > > > > > > const)
    itk::simple::TransformixImageFilter::SetTransformParameterMap(std::map< std::string,std::vector< std::string,std::allocator< std::string > >,std::less< std::string >,std::allocator< std::pair< std::string const,std::vector< std::string,std::allocator< std::string > > > > > const)


In [34]:
t = transformations[0].GetTransformParameterMap()
i = transformations[0]

In [38]:
i.WriteParameterFile(t[0], "/mnt/DATASETS/LUNA/parameter_map_0.tf")
i.WriteParameterFile(t[1], "/mnt/DATASETS/LUNA/parameter_map_1.tf")
i.WriteParameterFile(t[2], "/mnt/DATASETS/LUNA/parameter_map_2.tf")

<SimpleITK.SimpleITK.ElastixImageFilter; proxy of <Swig Object of type 'itk::simple::ElastixImageFilter::Self *' at 0x7f383f8dec30> >

In [32]:
transformixFilter = sitk.TransformixImageFilter()
#transformixFilter.SetMovingImage(reference_img)
transformixFilter.SetTransformParameterMap(transformations[0].GetTransformParameterMap())
transformixFilter.SetComputeDeformationField(True)
transformixFilter.Execute()

RuntimeError: Exception thrown in SimpleITK TransformixImageFilter_Execute: /simpleelastix/Code/Elastix/src/sitkTransformixImageFilterImpl.cxx:109:
sitk::ERROR: 
itk::ExceptionObject (0x1a915200)
Location: "unknown" 
File: /simpleelastix/build/Elastix/Core/Main/elxTransformixFilter.hxx
Line: 218
Description: itk::ERROR: Self(0x340d7a0): Internal transformix error: See transformix log (use LogToConsoleOn() or LogToFileOn())



In [31]:
transformixFilter.GetResultImage()

<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x7f3830ff0030> >

In [21]:
%time reference_img_mask = calculate_average_image_transformation(reference_img_mask, transformations)

CPU times: user 2min 25s, sys: 650 ms, total: 2min 26s
Wall time: 47.9 s


False